# Import

In [27]:
import os
from itertools import product
import pandas as pd

from sklearn.model_selection import (
    train_test_split,
    GridSearchCV
)

from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import (
    accuracy_score,
    precision_recall_fscore_support
)

import mlflow

# Load Dataset

In [4]:
data = pd.read_csv(os.path.join(os.getcwd(), 'data', 'dataset_processed.csv'))
data.head()

,Age,SibSp,Parch,Fare,Pclass_1,Pclass_2,Pclass_3,Sex_female,Sex_male,Embarked_C,Embarked_N,Embarked_Q,Embarked_S,Survived
0,0.271174,0.125,0.0,0.014151,0,0,1,0,1,0,0,0,1,0
1,0.472229,0.125,0.0,0.139136,1,0,0,1,0,1,0,0,0,1
2,0.321438,0.000,0.0,0.015469,0,0,1,1,0,0,0,0,1,1
3,0.434531,0.125,0.0,0.103644,1,0,0,1,0,0,0,0,1,1
4,0.434531,0.000,0.0,0.015713,0,0,1,0,1,0,0,0,1,0


In [5]:
target = data["Survived"]
data = data.drop("Survived", axis=1)

# Split Dataset

In [6]:
train_input, test_input, train_target, test_target = train_test_split(
    data, target,
    test_size=0.2,
    random_state=42
)

# Model Trainig Using 'MLFLOW Tracking'

- `custom logging` 활용 모델 학습 기록 관리
- `auto logging` 활용 모델 학습 기록 관리
- `auto logging` + `custom logging`

## `Custom Logging`

1. `tracking uri` 세팅 : `local host`로 지정
2. `experiment` 생성 (각 실험 프로젝트 별 관리를 위한 단위)
3. `experiment` 세팅

In [14]:
# set tracking uri (localhost web)
# mlflow.set_tracking_uri("http://our-team-uri...") -> for team mlflow tracking uri
mlflow.set_tracking_uri("http://127.0.0.1:5000")

# create experiment
mlflow.create_experiment("TitanicProject")

# set experiment
mlflow.set_experiment("TitanicProject")

<Experiment: artifact_location='mlflow-artifacts:/758022013946411100', creation_time=1755593227282, experiment_id='758022013946411100', last_update_time=1755593227282, lifecycle_stage='active', name='TitanicProject', tags={}>

mlflow를 활용한 Custom Logging

1. `mlflow.log_param()`
    - `n_estimator`
2. `mlflow.flog_model`
    - `save model`
3. `mlflow.log_metric()`
    - `metric` -> accuracy, precision, recall, f1score

In [15]:
param_set = {
    "n_estimator" : 80,
    "random_state" : 42
}

with mlflow.start_run():
    model = RandomForestClassifier(
        n_estimators=param_set['n_estimator'],
        random_state=param_set['random_state']
    )
    model.fit(train_input, train_target)

    preds = model.predict(test_input)

    accuracy = accuracy_score(test_target, preds)
    prf = precision_recall_fscore_support(test_target, preds, average='binary')

    mlflow.log_param("n_estimator", param_set['n_estimator'])
    mlflow.log_param("random_state", param_set['random_state'])
    mlflow.log_metric("accuracy_on_test", accuracy)
    mlflow.log_metric("precision_on_test", prf[0])
    mlflow.log_metric("recall_on_test", prf[1])
    mlflow.log_metric("f1score_on_test", prf[2])
    mlflow.sklearn.log_model(model, "model")

2025/08/19 17:55:53 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/08/19 17:55:54 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run abundant-mule-965 at: http://127.0.0.1:5000/#/experiments/758022013946411100/runs/385d5eb8ab4746099afa088787c39dea
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/758022013946411100


## `auto logging`

In [ ]:
mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.sklearn.autolog()

model = RandomForestClassifier(
        n_estimators=param_set['n_estimator'],
        random_state=param_set['random_state']
    )
model.fit(train_input, train_target)

preds = model.predict(test_input)

accuracy = accuracy_score(test_target, preds)
prf = precision_recall_fscore_support(test_target, preds, average='binary')

2025/08/19 17:59:21 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID 'a64166d34fb44512bb7886005a5d0c2a', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current sklearn workflow
2025/08/19 17:59:21 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/opt/anaconda3/envs/mlops/lib/python3.10/site-packages/mlflow/types/utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing 

🏃 View run defiant-auk-976 at: http://127.0.0.1:5000/#/experiments/758022013946411100/runs/a64166d34fb44512bb7886005a5d0c2a
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/758022013946411100


🏃 View run wise-loon-16 at: http://127.0.0.1:5000/#/experiments/758022013946411100/runs/71d9071bc40b4adf93e0addca038f8ba
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/758022013946411100


## `auto logging` + `custom logging`

- `auto logging`에서 수집되지 않는 정보 추가로 logging 하기

In [19]:
mlflow.set_tracking_uri("http://127.0.0.1:5000")

with mlflow.start_run():
    mlflow.sklearn.autolog()

    model = RandomForestClassifier(
            n_estimators=param_set['n_estimator'],
            random_state=param_set['random_state']
        )
    model.fit(train_input, train_target)

    preds = model.predict(test_input)

    accuracy = accuracy_score(test_target, preds)
    prf = precision_recall_fscore_support(test_target, preds, average='binary')

    mlflow.log_param("n_estimator", param_set['n_estimator'])
    mlflow.log_param("random_state", param_set['random_state'])
    mlflow.log_metric("accuracy_on_test", accuracy)
    mlflow.log_metric("precision_on_test", prf[0])
    mlflow.log_metric("recall_on_test", prf[1])
    mlflow.log_metric("f1score_on_test", prf[2])

2025/08/19 18:05:37 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/opt/anaconda3/envs/mlops/lib/python3.10/site-packages/mlflow/types/utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details."
2025/08/19 18:05:37 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/opt/anaconda3/envs/mlops/lib/python3.10/si

🏃 View run orderly-loon-798 at: http://127.0.0.1:5000/#/experiments/758022013946411100/runs/bf2b9ae6a17546b999053d0daa5057a6
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/758022013946411100


# Model Optimization using `MLFlow Tracking`

- 직접 parameter 변경하며 최적화
- GridSearch or RandomSearch 활용 모델 최적화

## 직접 변경

> 각 실험 결과를 하나의 experiment 항목으로 따로 보여줌 (실험이 많아지면 보기 어려워짐)

In [25]:
def train_model_with_param_set(param_set:dict, data_set:dict):
    with mlflow.start_run():
        model = RandomForestClassifier(
            n_estimators=param_set["n_estimator"],
            max_depth=param_set["max_depth"],
            max_features=param_set["max_features"]
        )
        model.fit(data_set["train_input"], data_set["train_target"])

        pred = model.predict(data_set["test_input"])

        accuracy = accuracy_score(data_set["test_target"], pred)
        pre_rec_f1 = precision_recall_fscore_support(data_set['test_target'], pred, average='binary')

        mlflow.log_metric("precision_on_test", pre_rec_f1[0])
        mlflow.log_metric("recall_on_test", pre_rec_f1[1])
        mlflow.log_metric("f1score_on_test", pre_rec_f1[2])
        mlflow.log_metric("accuracy_on_test", accuracy)

In [26]:
param_grid = {
    "n_estimator" : [50, 100, 150, 200],
    "max_depth" : [2, 5, 10],
    "max_features" : [5, 8, 10, 13]
}

for n_estimator, max_depth, max_features in product(
    param_grid["n_estimator"], 
    param_grid["max_depth"], 
    param_grid["max_features"]
    ):
    
    train_model_with_param_set(
        param_set={
            "n_estimator" : n_estimator,
            "max_depth" : max_depth,
            "max_features" : max_features
        },
        data_set = {
            "train_input" : train_input,
            "test_input" : test_input,
            "train_target" : train_target,
            "test_target" : test_target
        }
    )

2025/08/19 22:22:37 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/opt/anaconda3/envs/mlops/lib/python3.10/site-packages/mlflow/types/utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details."
2025/08/19 22:22:37 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/opt/anaconda3/envs/mlops/lib/python3.10/si

🏃 View run amusing-turtle-823 at: http://127.0.0.1:5000/#/experiments/758022013946411100/runs/d32097d7656b4cb784708c90f0282901
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/758022013946411100


2025/08/19 22:22:40 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/opt/anaconda3/envs/mlops/lib/python3.10/site-packages/mlflow/types/utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details."
2025/08/19 22:22:40 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/opt/anaconda3/envs/mlops/lib/python3.10/si

🏃 View run fearless-vole-300 at: http://127.0.0.1:5000/#/experiments/758022013946411100/runs/4a30678aa2c04cb683ccc9dd3ce5b103
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/758022013946411100


2025/08/19 22:22:42 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/opt/anaconda3/envs/mlops/lib/python3.10/site-packages/mlflow/types/utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details."
2025/08/19 22:22:42 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/opt/anaconda3/envs/mlops/lib/python3.10/si

...

🏃 View run sassy-eel-887 at: http://127.0.0.1:5000/#/experiments/758022013946411100/runs/0c48b8636a5b47ed8643e26a8138d332
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/758022013946411100


2025/08/19 22:23:57 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/opt/anaconda3/envs/mlops/lib/python3.10/site-packages/mlflow/types/utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details."
2025/08/19 22:23:57 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/opt/anaconda3/envs/mlops/lib/python3.10/si

🏃 View run hilarious-whale-355 at: http://127.0.0.1:5000/#/experiments/758022013946411100/runs/3bcb9b90f735422baa653bd1c88b8600
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/758022013946411100


2025/08/19 22:23:59 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/opt/anaconda3/envs/mlops/lib/python3.10/site-packages/mlflow/types/utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details."


🏃 View run chill-asp-883 at: http://127.0.0.1:5000/#/experiments/758022013946411100/runs/28cec361cdd64622a1b549588ec44119
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/758022013946411100


## GridSearch

> experiment에서 하나의 항목 아래에 포함되어 나타남 (확인이 용이)

In [29]:
param_grid = {
    "n_estimators" : [50, 100, 150, 200],
    "max_depth" : [2, 5, 10],
    "max_features" : [5, 8, 10, 13]
}

with mlflow.start_run():
    grid_tunor = GridSearchCV(RandomForestClassifier(), param_grid=param_grid)
    grid_tunor.fit(train_input, train_target)

    best_model = grid_tunor.best_estimator_

    pred = best_model.predict(test_input)

    accuracy = accuracy_score(test_target, pred)
    pre_rec_f1 = precision_recall_fscore_support(test_target, pred, average='binary')

    mlflow.log_metric("precision_on_test", pre_rec_f1[0])
    mlflow.log_metric("recall_on_test", pre_rec_f1[1])
    mlflow.log_metric("f1score_on_test", pre_rec_f1[2])
    mlflow.log_metric("accuracy_on_test", accuracy)

2025/08/19 22:30:18 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/opt/anaconda3/envs/mlops/lib/python3.10/site-packages/mlflow/types/utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details."
2025/08/19 22:30:33 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/opt/anaconda3/envs/mlops/lib/python3.10/si

🏃 View run puzzled-gull-635 at: http://127.0.0.1:5000/#/experiments/758022013946411100/runs/fbe307fa5e8d4d2ead2325a51669d021
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/758022013946411100
🏃 View run rebellious-roo-511 at: http://127.0.0.1:5000/#/experiments/758022013946411100/runs/c90e99ded7674703aa2b30294e0bbb77
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/758022013946411100
🏃 View run crawling-shark-870 at: http://127.0.0.1:5000/#/experiments/758022013946411100/runs/790f5a1b72674464ba131ff3c490b66b
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/758022013946411100
🏃 View run traveling-slug-412 at: http://127.0.0.1:5000/#/experiments/758022013946411100/runs/717cc648dff4435586bfb449ffa0cbb4
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/758022013946411100
🏃 View run sneaky-sheep-106 at: http://127.0.0.1:5000/#/experiments/758022013946411100/runs/e53b809e14d04b66ac1e2cc0dd9d483f
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/758